<a href="https://colab.research.google.com/github/Chanwoong1/recommendation-system-study/blob/main/%EC%BB%A8%ED%85%90%EC%B8%A0_%EA%B8%B0%EB%B0%98_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 컨텐츠 기반 모델
사용자가 이전에 구매한 상품 중에서 좋아하는 상품과 유사한 상품들을 추천하는 방법

텍스트와 같은 자연어는 TF-IDF, Word2Vec 등과 같은 기법을 사용하고 이미지는 CNN과 같은 모델을 사용한다. 

그리고 카카오 페이지의 소설책과 같은 경우는 표지의 이미지는 CNN 계열의 모델을 사용하고 내용물은 자연어 처리 모델을 같이 사용해서 결합하기도 한다.

# 유사도 함수
## 유클리디안 유사도
- 문서간의 유사도 계산

장점
- 계산하기 쉬움

단점
- 두 분포가 다르거나 범위가 다른 경우 상관성을 놓침

## 코사인 유사도
- 문서간의 유사도 계산

장점
- 벡터의 크기가 중요하지 않은 경우 거리를 측정하기 위한 방법으로 사용. 문서들의 길이가 비슷하지 않더라도 문서 내에서 얼마나 나왔는지 비율을 확인하기 때문에 상관이 없다.

단점
- 벡터의 크기가 중요한 경우 잘 작동하지 않음.

# 컨텐츠 기반의 추천시스템 - TF-IDF
TF-IDF는 특정 문서 내에 특정 단어가 얼마나 자주 등장하는 지를 의미하는 단어 빈도(TF)와 전체 문서에서 특정 단어가 얼마나 자주 등장하는지를 의미하는 역문서 빈도(DF)를 통해서 “다른 문서에서는 등장하지 않지만 특정 문서에서만 자주 등장하는 단어＂를 찾아서 문서 내 단어의 가중치를 계산하는 방법이다.

문서의 핵심어를 추출, 문서들 사이의 유사도 계산, 검색 결과의 중요도를 정하는 작업 등에 활용

- TF : 특정 문서 d에서의 특정 단어 t의 등장 횟수
- DF : 특정 단어 t가 등장한 문서의 수
- IDF : DF에 반비례하는 수 (idf-smoothing 여부에 따라서 결과가 달라짐)
- TF-IDF : TF와 IDF를 곱해준 값

## TF-IDF를 사용하는 이유

- Item이라는 컨텐츠를 벡터로 "Feature Extract" 과정을 수행해준다.
- 빈도수를 기반으로 많이 나오는 중요한 단어들을 잡아준다. 이러한 방법을 Counter Vectorizer라고 한다.
- 하지만, Counter Vectorizer는 단순 빈도만을 계산하기에 조사, 관사처럼 의미는 없지만 문장에 많이 등장하는 단어들도 높게 쳐주는 한계가 있다. 이러한 단어들에는 페널티를 줘서 적절하게 중요한 단어만을 잡아내는 게 TF-IDF 기법이다.


## 평가함수

평가 함수는 추천시스템의 모델을 생성하고 해당 모델이 얼마나 잘 추천하고 있는지에 대해서 평가를 도와주는 함수. 도메인이나 목적에 따라서 다른 평가 함수를 도입해서 얼마나 잘 추천이 되는지 평가하는 게 중요하다. 예를 들어, 영화 평점의 경우에서는 두 가지 형태로 평가를 할 수 있다.

- 내가 추천해준 영화를 고객이 봤나?
- 내가 추천해준 영화를 고객이 높은 점수로 평점을 줬나?

분명 위의 2가지는 다르며, 1번의 경우 단순히 보기만 하면 추천에 성공했다고 하지만, 실제 고객의 만족도는 낮을 수 있다. 반대로 2의 경우는 고객의 만족도까지 고려해서 평가를 한 것이다. 이러한 성질은 추천을 진행할 때에도 차이가 발생한다.

# TF-IDF 실습

In [1]:
docs = [
  '먹고 싶은 사과', # 문서0 
  '먹고 싶은 바나나', # 문서1
  '길고 노란 바나나 바나나', # 문서2 
  '저는 과일이 좋아요' # 문서3 
]

## CounterVectorizer

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() # Counter Vectorizer 객체 생성

In [3]:
# 문장을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(docs) 
countvect # 4x9 : 4개의 문서에 9개의 단어 

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [4]:
# toarray()를 통해서 문장이 Vector 형태의 값을 얻을 수 있음 
# 하지만, 각 인덱스와 컬럼이 무엇을 의미하는지에 대해서는 알 수가 없음 
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

In [5]:
# dictionary 형태의 값을 가지며, 각 value가 의미하는 게 칼럼의 위치가 되고 key는 해당 컬럼의 단어를 의미
vect.vocabulary_

{'과일이': 0,
 '길고': 1,
 '노란': 2,
 '먹고': 3,
 '바나나': 4,
 '사과': 5,
 '싶은': 6,
 '저는': 7,
 '좋아요': 8}

In [6]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [7]:
# 위의 Data Frame 형태의 유사도를 계산 
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

## TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)

In [9]:
tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [10]:
#  Sklearn의 TF-IDF는 IDF를 계산할 때 +1 부분이 분모에 있는 게 아니라 log의 밖에서 더해주기 때문에 계산이 조금 달라진다.
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidv_df, tfidv_df)

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

추가적으로, TF-IDF와 CounterVectorizer 모두 문서 내에 등장한 모든 단어들을 가지고 벡터를 만드는 특징이 있다. 

그렇기에, 문서가 많아지면 많아질수록 전체 단어가 많아질수록 벡터의 차원은 커지고 행렬 또한 감당할 수 없을 정도가 될 것이다. 이러한 문제를 방지해주기 위해서, 가장 많이 나온 단어 n 개만 사용하라는 `max_features` 파라미터들이 존재한다.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=4)
tfvect = vect.fit(docs)

tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,먹고,바나나,싶은
문서1,0.0,0.707107,0.00000,0.707107
문서2,0.0,0.577350,0.57735,0.577350
문서3,0.0,0.000000,1.00000,0.000000
문서4,1.0,0.000000,0.00000,0.000000
